In [17]:
import pandas as pd
from utils import get_repo_root, bucketize_sum_month_lags
from functools import reduce

In [4]:
repo_root = get_repo_root()

In [5]:
cust_hist = pd.read_csv(str(repo_root) + '/data/customer_history.csv')
cust = pd.read_csv(str(repo_root) + '/data/customers.csv')
ref_test = pd.read_csv(str(repo_root) + '/data/referance_data_test.csv')
ref = pd.read_csv(str(repo_root) + '/data/referance_data.csv')
sample_sub = pd.read_csv(str(repo_root) + '/data/sample_submission.csv')

Customer Dataset Feature Engineering

In [6]:
cust

,cust_id,gender,age,province,religion,work_type,work_sector,tenure
0,0,F,64,NOH,U,Part-time,Technology,135
1,1,F,57,ZUI,O,Full-time,Finance,65
2,2,F,62,NOB,M,Self-employed,Healthcare,224
3,3,F,22,ZUI,C,Student,NaN,47
4,5,M,27,ZUI,U,Full-time,Finance,108
...,...,...,...,...,...,...,...,...
176288,199995,F,54,GEL,C,Part-time,Public Sector,217
176289,199996,M,47,GEL,C,Full-time,Public Sector,37
176290,199997,F,66,NOB,C,Retired,NaN,227
176291,199998,F,31,ZUI,U,Self-employed,Education,156


In [7]:
cust_encoded = pd.get_dummies(cust, columns=['gender', 'work_type', 'work_sector', 'religion', 'province'], drop_first=True, dtype=int)

In [8]:
cust_encoded

,cust_id,age,tenure,gender_M,work_type_Part-time,work_type_Retired,work_type_Self-employed,work_type_Student,work_type_Unemployed,work_sector_Finance,...,province_FRI,province_GEL,province_GRO,province_LIM,province_NOB,province_NOH,province_OVE,province_UTR,province_ZEE,province_ZUI
0,0,64,135,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,57,65,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,2,62,224,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,3,22,47,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,27,108,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176288,199995,54,217,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
176289,199996,47,37,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
176290,199997,66,227,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
176291,199998,31,156,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Customer History Dataset Feature Engineering

In [9]:
ref_all = pd.concat([ref, ref_test], ignore_index=True)
ref_all

,cust_id,ref_date,churn
0,0,2017-09-01,0.0
1,3,2018-10-01,0.0
2,5,2018-03-01,1.0
3,6,2018-04-01,1.0
4,7,2018-05-01,0.0
...,...,...,...
176288,199951,2019-03-01,NaN
176289,199952,2019-05-01,NaN
176290,199963,2019-05-01,NaN
176291,199964,2019-03-01,NaN


In [10]:
cust_hist_ref_dates = cust_hist.merge(ref_all[['cust_id', 'ref_date']], on='cust_id', how='left')

In [11]:
cust_hist_ref_dates.head(5)

,cust_id,date,mobile_eft_all_cnt,active_product_category_nbr,mobile_eft_all_amt,cc_transaction_all_amt,cc_transaction_all_cnt,ref_date
0,0,2016-01-01,1.0,2,151.20,NaN,NaN,2017-09-01
1,0,2016-02-01,1.0,2,178.70,NaN,NaN,2017-09-01
2,0,2016-03-01,2.0,2,37.38,NaN,NaN,2017-09-01
3,0,2016-04-01,4.0,2,100.90,NaN,NaN,2017-09-01
4,0,2016-05-01,3.0,3,132.28,NaN,NaN,2017-09-01


In [21]:
numeric_features = [
    'mobile_eft_all_cnt',
    'mobile_eft_all_amt',    
    'cc_transaction_all_amt',
    'cc_transaction_all_cnt',
]

buckets = {
    "l0m": (0,0),
    "l2m": (1,2),
    "l5m": (3,5),
    "l8m": (6,8),
    "l1year": (9,12),
    "l2years": (13,24),
    "l4years": (25,48),
}

feature_tables = []
for col in numeric_features:
    feature_tables.append(
        bucketize_sum_month_lags(
            table=cust_hist_ref_dates,
            value_col=col,
            buckets=buckets,
        )
    )
features_merged = reduce(
    lambda left, right: pd.merge(
        left,
        right,
        on=["cust_id", "ref_date"],
        how="outer"
    ),
    feature_tables
)

In [22]:
features_merged

,cust_id,ref_date,mobile_eft_all_cnt_l0m,mobile_eft_all_cnt_l2m,mobile_eft_all_cnt_l5m,mobile_eft_all_cnt_l8m,mobile_eft_all_cnt_l1year,mobile_eft_all_cnt_l2years,mobile_eft_all_cnt_l4years,mobile_eft_all_amt_l0m,...,cc_transaction_all_amt_l1year,cc_transaction_all_amt_l2years,cc_transaction_all_amt_l4years,cc_transaction_all_cnt_l0m,cc_transaction_all_cnt_l2m,cc_transaction_all_cnt_l5m,cc_transaction_all_cnt_l8m,cc_transaction_all_cnt_l1year,cc_transaction_all_cnt_l2years,cc_transaction_all_cnt_l4years
0,0,2017-09-01,1.0,4.0,7.0,5.0,13.0,17.0,0.0,212.75,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-02-01,0.0,1.0,8.0,3.0,4.0,23.0,22.0,0.00,...,826.43,5001.60,1465.50,8.0,22.0,40.0,22.0,21.0,235.0,195.0
2,2,2019-01-01,1.0,5.0,7.0,3.0,5.0,32.0,34.0,810.71,...,51.20,59.13,93.56,7.0,8.0,9.0,9.0,11.0,10.0,19.0
3,3,2018-10-01,2.0,2.0,3.0,6.0,9.0,16.0,19.0,498.99,...,4306.40,7505.34,3216.61,21.0,37.0,62.0,90.0,156.0,321.0,148.0
4,5,2018-03-01,4.0,9.0,10.0,10.0,14.0,17.0,5.0,723.01,...,54.65,978.28,1311.10,3.0,5.0,8.0,8.0,12.0,222.0,95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176288,199995,2018-09-01,5.0,11.0,10.0,3.0,4.0,3.0,10.0,636.55,...,179.09,302.54,4575.49,4.0,141.0,257.0,172.0,81.0,55.0,346.0
176289,199996,2018-06-01,1.0,7.0,9.0,5.0,7.0,25.0,6.0,76.94,...,65.78,146.58,35.21,4.0,5.0,4.0,8.0,8.0,25.0,5.0
176290,199997,2018-12-01,4.0,8.0,15.0,21.0,46.0,187.0,45.0,1407.68,...,584.61,5473.04,1850.94,10.0,12.0,14.0,21.0,37.0,240.0,204.0
176291,199998,2018-02-01,6.0,14.0,11.0,15.0,17.0,28.0,2.0,566.82,...,58.94,1120.94,282.27,34.0,94.0,66.0,26.0,10.0,261.0,27.0
